In [3]:
import pandas as pd
import glob
import os

data_path = "../data"     
output_path = "../outputs"

files = sorted(glob.glob(os.path.join(data_path, "amazon_india_20*.csv")))
print("Found files:", files)

dfs = []
for f in files:
    print("Reading:", f)
    dfs.append(pd.read_csv(f, low_memory=False))
    
df = pd.concat(dfs, ignore_index=True)
print("Combined dataset shape:", df.shape)

os.makedirs(output_path, exist_ok=True)
df.to_csv(os.path.join(output_path, "amazon_india_2015_2025_raw.csv"), index=False)
print("Combined file saved to outputs/amazon_india_2015_2025_raw.csv")


Found files: ['../data\\amazon_india_2015.csv', '../data\\amazon_india_2016.csv', '../data\\amazon_india_2017.csv', '../data\\amazon_india_2018.csv', '../data\\amazon_india_2019.csv', '../data\\amazon_india_2020.csv', '../data\\amazon_india_2021.csv', '../data\\amazon_india_2022.csv', '../data\\amazon_india_2023.csv', '../data\\amazon_india_2024.csv', '../data\\amazon_india_2025.csv']
Reading: ../data\amazon_india_2015.csv
Reading: ../data\amazon_india_2016.csv
Reading: ../data\amazon_india_2017.csv
Reading: ../data\amazon_india_2018.csv
Reading: ../data\amazon_india_2019.csv
Reading: ../data\amazon_india_2020.csv
Reading: ../data\amazon_india_2021.csv
Reading: ../data\amazon_india_2022.csv
Reading: ../data\amazon_india_2023.csv
Reading: ../data\amazon_india_2024.csv
Reading: ../data\amazon_india_2025.csv
Combined dataset shape: (1127609, 34)
Combined file saved to outputs/amazon_india_2015_2025_raw.csv


In [4]:
import pandas as pd
import sys
import os
sys.path.append('../src')
from cleaning_utils import * 
df = pd.read_csv("../outputs/amazon_india_2015_2025_raw.csv", low_memory=False)
print("Initial rows:", len(df))
df.head(3)


Initial rows: 1127609


,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,...,is_festival_sale,festival_name,customer_rating,return_status,order_month,order_year,order_quarter,product_weight_kg,is_prime_eligible,product_rating
0,TXN_2015_00000001,2015-01-25,CUST_2015_00003884,PROD_000021,Samsung Galaxy S6 16GB Black,Electronics,Smartphones,Samsung,123614.29,27.91,...,True,Republic Day Sale,5.0,Delivered,1,2015,1,0.19,True,4.7
1,TXN_2015_00000002,2015-01-05,CUST_2015_00011709,PROD_000055,OnePlus OnePlus 2 16GB White,Electronics,Smartphones,OnePlus,54731.86,0.00,...,False,NaN,4.5,Delivered,1,2015,1,0.20,True,4.1
2,TXN_2015_00000003,2015-01-24,CUST_2015_00004782,PROD_000039,Samsung Galaxy Note 5 64GB Black,Electronics,Smartphones,Samsung,97644.25,46.93,...,True,Republic Day Sale,NaN,Delivered,1,2015,1,0.17,True,3.3


In [8]:
# Dates
df = clean_order_date(df, 'order_date')

# Price
df = clean_original_price(df, 'original_price_inr')

# Ratings
df = clean_ratings(df, 'product_rating')

# City names
df = clean_customer_city(df, 'customer_city')

# Boolean columns
df = clean_boolean_cols(df, ['is_prime_member','is_prime_eligible','is_festival_sale'])

# Categories
df = clean_category(df, 'category')

# Delivery days
df = clean_delivery_days(df, 'delivery_days')

# Duplicates
df = mark_duplicates(df)

# Outlier correction (depends on category_clean)
df = correct_price_outliers(df, price_col='original_price_inr_clean')

# Payment methods
df = clean_payment_method(df, 'payment_method')

print("All cleaning functions applied!")


All cleaning functions applied!


In [10]:
df[['order_date', 'order_date_clean']].sample(5)



,order_date,order_date_clean
574576,2021-05-02,2021-05-02
766207,2022-10-29,2022-10-29
1044537,21/12/2024,2024-12-21
331862,2019-08-28,2019-08-28
832048,2023-04-14,2023-04-14


In [11]:
df[['original_price_inr', 'original_price_inr_clean']].sample(5)


,original_price_inr,original_price_inr_clean
600513,125574.02,125574.02
477895,98321.19,98321.19
1096616,27725.18,27725.18
922307,181191.8,181191.80
944163,111414.43,111414.43


In [12]:
df[['product_rating', 'product_rating_clean']].sample(5)


,product_rating,product_rating_clean
590767,4.4,4.4
462574,4.4,4.4
486099,4.6,4.6
374731,3.5,3.5
759510,4.5,4.5


In [13]:
cols_to_replace = {
    'order_date_clean': 'order_date',
    'original_price_inr_clean': 'original_price_inr',
    'product_rating_clean': 'product_rating',
    'customer_city_clean': 'customer_city',
    'category_clean': 'category',
    'delivery_days_clean': 'delivery_days',
    'payment_method_clean': 'payment_method'
}
for clean_col, orig_col in cols_to_replace.items():
    if clean_col in df.columns:
        df[orig_col] = df[clean_col]
        df.drop(columns=[clean_col], inplace=True, errors='ignore')

print("Columns replaced. Current shape:", df.shape)


Columns replaced. Current shape: (1127609, 41)


In [14]:
os.makedirs("../outputs", exist_ok=True)
df.to_csv("../outputs/amazon_cleaned.csv", index=False)
df.to_parquet("../outputs/amazon_cleaned.parquet", index=False)
print("Cleaned dataset saved to outputs folder!")

Cleaned dataset saved to outputs folder!


In [15]:
print("Total rows:", len(df))
print("Columns:", len(df.columns))
print("\nMissing values per column (top 10):")
print(df.isna().sum().sort_values(ascending=False).head(10))
print("\nDuplicate rows:", df.duplicated().sum())


Total rows: 1127609
Columns: 41

Missing values per column (top 10):
festival_name         777736
customer_rating       341696
customer_age_group    135315
delivery_charges       90201
delivery_days           2317
order_date                 0
customer_id                0
product_id                 0
transaction_id             0
original_price_inr         0
dtype: int64

Duplicate rows: 0


In [16]:
print("Columns replaced. Current shape:", df.shape)

Columns replaced. Current shape: (1127609, 41)


In [17]:
print("Cleaned dataset successfully created.")
print("Shape:", df.shape)
print("\nMissing values per column (top 10):")
print(df.isna().sum().sort_values(ascending=False).head(10))
print("\nDuplicate rows:", df.duplicated().sum())

print("\nPrice range:", df['original_price_inr'].min(), "→", df['original_price_inr'].max())
print("Rating range:", df['product_rating'].min(), "→", df['product_rating'].max())

print("\nUnique payment methods:", df['payment_method'].dropna().unique()[:10])
print("Unique categories:", df['category'].dropna().unique()[:10])
print("Unique cities:", df['customer_city'].dropna().unique()[:10])


Cleaned dataset successfully created.
Shape: (1127609, 41)

Missing values per column (top 10):
festival_name         777736
customer_rating       341696
customer_age_group    135315
delivery_charges       90201
delivery_days           2317
order_date                 0
customer_id                0
product_id                 0
transaction_id             0
original_price_inr         0
dtype: int64

Duplicate rows: 0

Price range: -300102.68 → 33371693.0
Rating range: 3.0 → 4.8

Unique payment methods: ['COD' 'Debit Card' 'Credit Card' 'Netbanking' 'UPI' 'Wallet' 'Bnpl']
Unique categories: ['Electronics']
Unique cities: ['Mumbai' 'Allahabad' 'Kolkata' 'Ludhiana' 'New Delhi' 'Lucknow' 'Jaipur'
 'Bhubaneswar' 'Ahmedabad' 'Bengaluru']


In [21]:
Advanced Data Cleaning — Q1 to Q10
====================================

Q1. Clean and standardize all order dates
Question: Your dataset contains order_date in multiple formats — 'DD/MM/YYYY', 'DD-MM-YY', 'YYYY-MM-DD', and invalid entries like '32/13/2020'. Clean and standardize all dates to 'YYYY-MM-DD' format, handling invalid dates appropriately.

Problem: The dataset includes inconsistent date formats and some impossible values. These make time-series analysis (like revenue by year or month) unreliable.

Approach: Used the function:
    
df = clean_order_date(df, 'order_date')
    
It applies parse_date_safe() which tries multiple date formats and uses dateutil.parser as a fallback. Invalid or unreadable dates are converted to NaT.
Result: All dates are now in consistent YYYY-MM-DD format. Invalid or corrupted entries are safely handled as missing (NaT).
    
========================================================================
=========================================================================  
    
Q2. Clean and standardize the original_price_inr column

Question: original_price_inr contains numeric values, text with ₹ symbols, comma separators (₹1,25,000), and entries like "Price on Request". Clean this column to contain only numeric values in Indian Rupees.

Problem: The column mixes text, currency symbols, and non-numeric entries which prevent correct numeric computations.

Approach: Used:
df = clean_original_price(df, 'original_price_inr')

The function removes currency symbols (₹), commas, and non-numeric text. Entries like "Price on Request" are replaced with NaN.

Result: All prices are stored as clean numeric values in INR, ready for statistical or financial analysis.
    
========================================================================
=========================================================================  
 
Q3. Standardize customer ratings

Question: Customer ratings appear as '5.0', '4 stars', '3/5', '2.5/5.0', and missing values. Standardize all ratings to a numeric 1.0–5.0 scale.

Problem: Mixed rating formats and text prevent accurate averaging and comparison.

Approach: Used:
df = clean_ratings(df, 'product_rating')
Extracts numeric parts from each string, converts percentages (like '80%') to a 1–5 scale, and caps all values at 5. Missing values remain NaN.

Result: All ratings are standardized numeric floats between 1.0 and 5.0.
    
========================================================================
=========================================================================  
 
Q4. Standardize city names

Question: The customer_city column has inconsistent naming: 'Bangalore/Bengaluru', 'Mumbai/Bombay', 'Delhi/New Delhi', along with spelling errors and case variations.

Problem: Multiple variations of the same city prevent grouping or geographic analysis.

Approach: Used:
df = clean_customer_city(df, 'customer_city')

Applies a city mapping (CITY_MAP) to merge variants like 'Bombay' → 'Mumbai', 'Bangalore' → 'Bengaluru', 'Delhi' → 'New Delhi'. Also fixes casing and typos.

Result: All city names are standardized to canonical forms (e.g., Bengaluru, Mumbai, New Delhi).
    
========================================================================
=========================================================================  
 
Q5. Convert Boolean columns to consistent format

Question: Boolean columns (is_prime_member, is_prime_eligible, is_festival_sale) contain mixed values like True/False, Yes/No, 1/0, Y/N, and missing entries.

Problem: Inconsistent boolean representation makes filtering and logic operations fail.

Approach: Used:

df = clean_boolean_cols(df, ['is_prime_member', 'is_prime_eligible', 'is_festival_sale'])

Converts all truthy/falsey values to True or False (Python boolean type).

Result: All three boolean columns are clean and consistent (True or False), with missing entries preserved as NaN.
    
========================================================================
=========================================================================  
 
Q6. Standardize product categories

Question: Product categories vary as 'Electronics/Electronic/ELECTRONICS/Electronics & Accessories'.

Problem: Different naming conventions and letter casing cause duplicate category entries.

Approach: Used:
df = clean_category(df, 'category')

Normalizes text (lowercase, replaces symbols, removes special characters). Then maps similar words to a common category name such as Electronics, Fashion, Home & Kitchen, etc.

Result: All product categories follow a consistent naming convention, allowing accurate grouping and comparison.
    
========================================================================
=========================================================================  
 
Q7. Clean the delivery_days column

Question: delivery_days has text like 'Same Day', '1-2 days', and unrealistic values like 50 or negative numbers.

Problem: Delivery time analysis becomes meaningless if entries aren’t numeric or valid.

Approach: Used:
df = clean_delivery_days(df, 'delivery_days')

Converts text to numeric (mean of ranges), 'Same Day' → 0, and removes invalid or unrealistic values (negative or > 30).

Result: All delivery_days values are numeric and realistic.
    
========================================================================
=========================================================================  
 
Q8. Identify and handle duplicate transactions

Question: Some transactions are exact duplicates of the same customer, product, date, and amount — some are genuine bulk orders, others are data errors.

Problem: Duplicate records distort revenue and customer counts.

Approach: Used:
 df = mark_duplicates(df)

Creates a composite key combining customer_id, product_id, order_date, and final_amount_inr. Marks potential duplicates with is_duplicate = True.

Result: Duplicate entries are identified for manual review or removal as needed.
    
========================================================================
=========================================================================  
 
Q9. Correct outlier prices

Question: Some products show prices 100× higher than expected due to misplaced decimal points.

Problem: Extreme price outliers distort average price and revenue calculations.

Approach: Used:
df = correct_price_outliers(df, price_col='original_price_inr_clean')

Compares each price with the category median. If the price is 20–100× higher, divides by 10 or 100 accordingly.

Result:Outlier prices corrected to realistic levels based on category norms.
    
========================================================================
=========================================================================  
 
Q10. Standardize payment method names

Question: Payment methods have inconsistent naming: 'UPI/PhonePe/GooglePay', 'Credit Card/CREDIT_CARD/CC', 'Cash on Delivery/COD/C.O.D'.

Problem: Variations make it difficult to analyze payment preferences.

Approach: Used:
df = clean_payment_method(df, 'payment_method')

Maps similar names into six standard groups:
UPI, Credit Card, Debit Card, Netbanking, COD, Wallet.

Result:Payment methods are standardized across all transactions, enabling clean analysis of payment trends.   

SyntaxError: invalid character '—' (U+2014) (3868511953.py, line 1)